<center>
<img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</center>
<center>
<h1>Ingeniería de Características</h1>
</center>
<center>
<h3>Tarea 1</h3>
</center>
<center>
<h3> Fernando Luna Ponce </h3>
</center>

Importamos las librerías necesarias

In [268]:
import requests
import xml.dom.minidom
import xml.etree.ElementTree as ET
import re
import pandas as pd

Definimos funciones que nos servirán más adelante

In [269]:
def getRegexText(regex, labelText): #Para obtener los textos de un string que hagan match con la expresión regular
  matches =  re.finditer(regex, labelText, re.MULTILINE) # regex multilinea
  for matchNum, match in enumerate(matches, start=1):
    texto = match.group()
    return texto

def get_namespace(element): # Para obtener el namespace
  m = re.match('\{.*\}', element.tag)
  return m.group(0) if m else ''

def getTagName(tag, namespace): # Para concatenar el namespace con el nombre del tag que queremos encontrar
  return namespace + tag

Hacemos la petición de descarga para el archivo solicitado y lo guardamos de manera local como xml

In [270]:
url = 'https://raw.githubusercontent.com/mcd-unison/ing-caract/main/ejemplos/integracion/ejemplos/wikipedia-poetas.xml'

response = requests.get(url)
with open('poetas.xml', 'wb') as file:
    file.write(response.content)


utilizamos las funciones de la biblioteca xml.dom.minidom para mostrar la información del xml estilo que es más fácil para visualizar su estructura

In [ ]:
dom = xml.dom.minidom.parse('poetas.xml') # xml.dom.minidom.parseString(xml_string)
pretty_xml_as_string = dom.toprettyxml()
print(pretty_xml_as_string)

Como podemos observar, se genera todo un árbol que presenta cierta estructura, aquí vemos que tenemos al mismo nivel la etiqueta siteinfo y page, de las cuales siteinfo no la necesitamos y también se tiene que por cada etiqueta page está contenida la información de un poeta argentino.

Extraemos la información del xml con las funciones de ElementTree

In [272]:
tree = ET.parse('poetas.xml')
root = tree.getroot()

Obtenemos el namespace del xml

In [273]:
namespace = get_namespace(tree.getroot()) # Obtenemos el namespace del xml
print(namespace)

{http://www.mediawiki.org/xml/export-0.10/}


Despues de analizar el xml hemos determinado que vamos a guardar a guardar el nombre de cada autor y las obras que realizó. Esta información nos pude servir para analizar la cantidad obras realizadas por cada autor y su contribución a la literatura argentina.

Vamos a guardar toda esta información en un dataframe, con un registro por cada obra. Cada registro contendrá un id incremental asignado a cada autor y un id incremental para cada obra.

In [274]:
df = pd.DataFrame(columns=['idAutor', 'Autor', 'IdObra', 'Obra'])

Accedemos a la información con un ciclo para extraer el nombre del autor, y utilizamos un ciclo for para extraer la información de sus obras, solo que en esta parte también necesitamos hacer uso de expresiones regulares para extraer el nombre de la obra.

In [275]:
regexObras = r"\={2,3}\sObras?\s\={2,3}\s(\n.|.)*[^\S]{2}" # Expresión regular para extraer solo la parte de obras de cada texto
regexEObra = r"^\*\s.*" # Expresión regular para extraer individualmente cada obra realizada por el autor

i = 0
j = 0

for page in root.findall(getTagName('page', namespace)):
    autor = page.find(getTagName('title', namespace))
    title =  page.find(getTagName('revision', namespace))
    texto = title.find(getTagName('text', namespace))
    Biografias = getRegexText(regexObras, texto.text)
    if Biografias != None: # Si el autor no tiene información 
        matches = re.finditer(regexEObra, Biografias, re.MULTILINE)
        for matchNum, match in enumerate(matches, start=1):
            b = match.group()
            df.loc[j] = [i, autor.text, matchNum-1, b]
            j += 1
    i += 1


Mostramos el contenido del dataframe generado.

In [276]:
print(df)

      idAutor                      Autor  IdObra  \
0           0  Julia Morilla de Campbell       0   
1           0  Julia Morilla de Campbell       1   
2           0  Julia Morilla de Campbell       2   
3           0  Julia Morilla de Campbell       3   
4           0  Julia Morilla de Campbell       4   
...       ...                        ...     ...   
1831      632                 Ireneo Paz       4   
1832      632                 Ireneo Paz       5   
1833      632                 Ireneo Paz       6   
1834      634            Humberto Tejera       0   
1835      634            Humberto Tejera       1   

                                                   Obra  
0     * 1976: ''Crines al viento''.<ref>[http://abr....  
1     * 1976: ''Adiós Canopus: viaje por la Mesopota...  
2     * 1978: ''Viajando con el calendario escolar: ...  
3     * 1978: ''El rey: aventuras de un cóndor neuqu...  
4     * 1976: ''Sombrero negro, vida y aventuras en ...  
...                        

Guardamos el dataframe en formato parquet

In [277]:
df.to_parquet('data.parquet')

Leemos el archivo para observar su estructura

In [278]:
parquet_file = 'data.parquet' 
pd.read_parquet(parquet_file, engine='pyarrow')

,idAutor,Autor,IdObra,Obra
0,0,Julia Morilla de Campbell,0,* 1976: ''Crines al viento''.<ref>[http://abr....
1,0,Julia Morilla de Campbell,1,* 1976: ''Adiós Canopus: viaje por la Mesopota...
2,0,Julia Morilla de Campbell,2,* 1978: ''Viajando con el calendario escolar: ...
3,0,Julia Morilla de Campbell,3,* 1978: ''El rey: aventuras de un cóndor neuqu...
4,0,Julia Morilla de Campbell,4,"* 1976: ''Sombrero negro, vida y aventuras en ..."
...,...,...,...,...
1831,632,Ireneo Paz,4,"* ''Leyendas históricas de la Independencia'',..."
1832,632,Ireneo Paz,5,"* ''Vida y aventuras de Joaquín Murrieta, famo..."
1833,632,Ireneo Paz,6,"* ''Leyendas históricas'', [[1914]]"
1834,634,Humberto Tejera,0,* ''Los Gómez y el poder judicial de Venezuela...
